In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from encode_data import *
from midi_data import *

In [3]:
from tqdm import tqdm

## Encode music21 stream to text representation 

This notebook uses a full component format 
- measure separators, instruments, and separated octaves
- Format: note, octave, action type, instrument
- note repr: nG# o4 t1 i0

### Load midi data

In [4]:
path = Path('data/midi')
csv_path = path/'metadata'

In [5]:
source_dir = 'midi_transform_v1'
out_dir = 'midi_transcribe_v1'
source_csv = csv_path/f'{source_dir}.csv'
out_csv = csv_path/f'{out_dir}.csv'

In [6]:
import pandas as pd

In [7]:
df = pd.read_csv(source_csv, index_col=0); df.head()

,artist,bpm,genres,ht_bpm,ht_key,ht_mode,ht_offset,ht_time_signature,inferred_key,inferred_offset,...,midi,midi_title,parts,seconds,section,song_url,source,time_signature,title,midi_transform_v1
0,wayne-sharpe,128.0,NaN,128.0,C,1.0,0.0,4.0,C major,0.0,...,data/midi/midi_sources/hooktheory/pianoroll/w/...,yu-gi-oh3,"intro,chorus",15.000000,chorus,https://www.hooktheory.com/theorytab/view/wayn...,hooktheory,4/4,yu-gi-oh-theme-song,data/midi/midi_transform_v1/classic_piano/chpn...
0,Tune Up,142.0,dance,NaN,NaN,NaN,NaN,NaN,F major,-5.0,...,data/midi/midi_sources/freemidi/genre-dance-el...,NaN,NaN,28.732394,NaN,NaN,freemidi,NaN,Bounce,data/midi/midi_transform_v1/classic_piano/chpn...
0,Kona,176.0,"pop,inferred",NaN,NaN,NaN,NaN,NaN,B minor,-2.0,...,data/midi/midi_sources/midiworld/named_midi/Ko...,NaN,NaN,1.193182,NaN,NaN,midiworld,4/4,Drumloop,data/midi/midi_transform_v1/classic_piano/chpn...
0,,117.0,classical,NaN,NaN,NaN,NaN,NaN,A minor,0.0,...,data/midi/midi_sources/ecomp/2017/WangH09.MID,NaN,NaN,212.820513,NaN,NaN,ecomp,4/4,II. Intermezzo in A Minor,data/midi/midi_transform_v1/classic_piano/chpn...
0,Alex Gaudino,143.0,"EDM,inferred",NaN,NaN,NaN,NaN,NaN,A- major,4.0,...,data/midi/midi_sources/cprato/Alex Gaudino - C...,NaN,NaN,28.531469,NaN,NaN,cprato,NaN,Calabria (Drunken Monkey Remix) (Midi By Carlo...,data/midi/midi_transform_v1/classic_piano/chpn...


In [8]:
df_filtered = df.loc[df['time_signature'] == '4/4']; df_filtered.shape

(30201, 21)

In [9]:
from data_sources import process_parallel

In [10]:
def transcribe_file(idxrow):
    idx,row = idxrow
    midi_file = row[source_dir]
    if not isinstance(midi_file, str) or not Path(midi_file).exists(): return idx,None
    out_file = Path(midi_file.replace(f'/{source_dir}/', f'/{out_dir}/')).with_suffix('.txt')
    out_file.parent.mkdir(parents=True, exist_ok=True)
    if out_file.exists(): return idx,str(out_file)
    try:
        seq = midi2seq(midi_file)
        seq_comp = remove_seq_rests(trim_seq_rests(seq))
        delta_trim = len(seq) - len(seq_comp)
        if delta_trim > 100: print(f'Removed {delta_trim} rests from {midi_file}')
        string_repr = seq2str(seq)
        with open(out_file, 'w') as tf:
            tf.write(string_repr)
    except Exception as e:
        print('Error converting midi to sequence', e)
        return idx,None
    return idx,str(out_file)

In [11]:
# for r in df_filtered.iterrows():
#     transcribe_file(r)

In [ ]:
process_parallel(transcribe_file, df_filtered.iterrows(), total=df_filtered.shape[0])

Removed 457 rests from data/midi/midi_transform_v1/classic_piano/bor_ps1_format0.mid
Removed 457 rests from data/midi/midi_transform_v1/classic_piano/bor_ps1_format0.mid
Removed 457 rests from data/midi/midi_transform_v1/classic_piano/bor_ps1_format0.mid
Removed 457 rests from data/midi/midi_transform_v1/classic_piano/bor_ps1_format0.mid
Removed 457 rests from data/midi/midi_transform_v1/classic_piano/bor_ps1_format0.mid
Removed 457 rests from data/midi/midi_transform_v1/classic_piano/bor_ps1_format0.mid


Removed 197 rests from data/midi/midi_transform_v1/cprato/Rogue - From The Dust  (midi by Carlo Prato) (www.cprato.com).mid
Removed 197 rests from data/midi/midi_transform_v1/cprato/Rogue - From The Dust  (midi by Carlo Prato) (www.cprato.com).mid
Removed 197 rests from data/midi/midi_transform_v1/cprato/Rogue - From The Dust  (midi by Carlo Prato) (www.cprato.com).mid
Removed 197 rests from data/midi/midi_transform_v1/cprato/Rogue - From The Dust  (midi by Carlo Prato) (www.cprato.com).mid
Removed 197 rests from data/midi/midi_transform_v1/cprato/Rogue - From The Dust  (midi by Carlo Prato) (www.cprato.com).mid
Removed 129 rests from data/midi/midi_transform_v1/cprato/deadmau5 - Deus Ex Machina  (midi by Carlo Prato) (www.cprato.com).mid
Removed 129 rests from data/midi/midi_transform_v1/cprato/deadmau5 - Deus Ex Machina  (midi by Carlo Prato) (www.cprato.com).mid
Removed 129 rests from data/midi/midi_transform_v1/cprato/deadmau5 - Deus Ex Machina  (midi by Carlo Prato) (www.cprato.co

Removed 246 rests from data/midi/midi_transform_v1/classic_piano/mz_570_3_format0.mid
Removed 125 rests from data/midi/midi_transform_v1/classic_piano/mendel_op53_5_format0.mid
Removed 129 rests from data/midi/midi_transform_v1/cprato/Aly & Fila Meets Roger Shah & Susana - Unbreakable  (midi by Carlo Prato) (www.cprato.com).mid
Removed 129 rests from data/midi/midi_transform_v1/cprato/Aly & Fila Meets Roger Shah & Susana - Unbreakable  (midi by Carlo Prato) (www.cprato.com).mid
Removed 107 rests from data/midi/midi_transform_v1/cprato/Aether & Enzalla - Elysia's Heart  (midi by Carlo Prato) (www.cprato.com).mid
Removed 107 rests from data/midi/midi_transform_v1/cprato/Aether & Enzalla - Elysia's Heart  (midi by Carlo Prato) (www.cprato.com).mid
Removed 129 rests from data/midi/midi_transform_v1/cprato/Aly & Fila Meets Roger Shah & Susana - Unbreakable  (midi by Carlo Prato) (www.cprato.com).mid
Removed 107 rests from data/midi/midi_transform_v1/cprato/Aether & Enzalla - Elysia's Heart 

Removed 145 rests from data/midi/midi_transform_v1/classic_piano/schub_d760_4_format0.mid
Removed 121 rests from data/midi/midi_transform_v1/classic_piano/mz_311_1_format0.mid
Removed 144 rests from data/midi/midi_transform_v1/classic_piano/rac_op23_5_format0.mid
Removed 144 rests from data/midi/midi_transform_v1/classic_piano/rac_op23_5_format0.mid
Removed 274 rests from data/midi/midi_transform_v1/classic_piano/mz_333_3_format0.mid
Compressing rests: 22 -> 18
Removed 211 rests from data/midi/midi_transform_v1/cprato/Kaaris - Chargé (Mr. Carmack Remix) (midi by Carlo Prato) (www.cprato.com).mid
Compressing rests: 22 -> 18
Removed 211 rests from data/midi/midi_transform_v1/cprato/Kaaris - Chargé (Mr. Carmack Remix) (midi by Carlo Prato) (www.cprato.com).mid
Compressing rests: 22 -> 18
Removed 211 rests from data/midi/midi_transform_v1/cprato/Kaaris - Chargé (Mr. Carmack Remix) (midi by Carlo Prato) (www.cprato.com).mid
Removed 274 rests from data/midi/midi_transform_v1/classic_piano/mz

Removed 2395 rests from data/midi/midi_transform_v1/classic_piano/deb_pass_format0.mid
Removed 2395 rests from data/midi/midi_transform_v1/classic_piano/deb_pass_format0.mid
Removed 423 rests from data/midi/midi_transform_v1/classic_piano/beethoven_opus22_1_format0.mid
Removed 423 rests from data/midi/midi_transform_v1/classic_piano/beethoven_opus22_1_format0.mid
Removed 423 rests from data/midi/midi_transform_v1/classic_piano/beethoven_opus22_1_format0.mid
Removed 423 rests from data/midi/midi_transform_v1/classic_piano/beethoven_opus22_1_format0.mid
Removed 423 rests from data/midi/midi_transform_v1/classic_piano/beethoven_opus22_1_format0.mid
Removed 220 rests from data/midi/midi_transform_v1/classic_piano/liz_et_trans4_format0.mid
Removed 220 rests from data/midi/midi_transform_v1/classic_piano/liz_et_trans4_format0.mid
Removed 220 rests from data/midi/midi_transform_v1/classic_piano/liz_et_trans4_format0.mid
Removed 544 rests from data/midi/midi_transform_v1/classic_piano/beethove

Removed 2124 rests from data/midi/midi_transform_v1/ecomp/2004/HONG02.mid
Removed 2124 rests from data/midi/midi_transform_v1/ecomp/2004/HONG02.mid
Removed 943 rests from data/midi/midi_transform_v1/ecomp/2004/EVSTIO02.mid
Removed 943 rests from data/midi/midi_transform_v1/ecomp/2004/EVSTIO02.mid
Removed 2124 rests from data/midi/midi_transform_v1/ecomp/2004/HONG02.mid
Removed 943 rests from data/midi/midi_transform_v1/ecomp/2004/EVSTIO02.mid
Removed 234 rests from data/midi/midi_transform_v1/classic_piano/mond_3_format0.mid
Removed 2938 rests from data/midi/midi_transform_v1/ecomp/2004/SUN01.mid
Removed 2938 rests from data/midi/midi_transform_v1/ecomp/2004/SUN01.mid
Removed 1072 rests from data/midi/midi_transform_v1/ecomp/2004/CHOE01.mid
Removed 2938 rests from data/midi/midi_transform_v1/ecomp/2004/SUN01.mid
Removed 2938 rests from data/midi/midi_transform_v1/ecomp/2004/SUN01.mid
Compressing rests: 26 -> 18
Compressing rests: 26 -> 18
Removed 1072 rests from data/midi/midi_transfor

Removed 203 rests from data/midi/midi_transform_v1/ecomp/2004/FONG09.mid
Removed 2287 rests from data/midi/midi_transform_v1/ecomp/2009/Goh04.MID
Compressing rests: 23 -> 19
Compressing rests: 23 -> 19
Removed 203 rests from data/midi/midi_transform_v1/ecomp/2004/FONG09.mid
Compressing rests: 23 -> 19
Removed 2287 rests from data/midi/midi_transform_v1/ecomp/2009/Goh04.MID
Removed 2692 rests from data/midi/midi_transform_v1/ecomp/2009/WongDoe01.MID
Removed 2287 rests from data/midi/midi_transform_v1/ecomp/2009/Goh04.MID
Compressing rests: 23 -> 19
Removed 2692 rests from data/midi/midi_transform_v1/ecomp/2009/WongDoe01.MID
Removed 935 rests from data/midi/midi_transform_v1/ecomp/2004/SCHU03.mid
Removed 203 rests from data/midi/midi_transform_v1/ecomp/2004/FONG09.mid
Removed 2692 rests from data/midi/midi_transform_v1/ecomp/2009/WongDoe01.MID
Removed 871 rests from data/midi/midi_transform_v1/ecomp/2009/Gokcin04.MID
Removed 871 rests from data/midi/midi_transform_v1/ecomp/2009/Gokcin04.

In [ ]:
tdf = pd.DataFrame(data={out_dir: list(transcribed_files.values())}, index=list(transcribed_files.keys()))

In [ ]:
merged_df = df.join(tdf, how='outer'); tdf.shape, df.shape, merged_df.shape

In [ ]:
merged_df.to_csv(out_csv); merged_df.head()